In [1]:
#getting OPEN AI KEY and ENDPOINT

from helper import get_openai_api_key

OPENAI_API_KEY = get_openai_api_key()

In [2]:
import nest_asyncio #is that jupyter runs an even loop behind the scenes

nest_asyncio.apply()

In [17]:
#LOAD DATA

from llama_index.core import SimpleDirectoryReader # use for read the pdf offered by llamaindex

# load documents
documents = SimpleDirectoryReader(input_files=["Design of a cooling system for a hybrid-powered vehicle.pdf"]).load_data()

In [ ]:
#Define LLM and Embedding model

In [4]:
from llama_index.core.node_parser import SentenceSplitter #split on the order of the sentences

splitter = SentenceSplitter(chunk_size=1024) #chunk size = 1024
nodes = splitter.get_nodes_from_documents(documents)

In [5]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding #use as a embedding

Settings.llm = OpenAI(model="gpt-3.5-turbo") #llm -> OpenAI ,model-> gpt-3.5-turbo
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002") # embedding model->OpenAIEmbedding

In [ ]:
#Define Summary Index and Vector Index over the Same Data and also have more explanation with code

In [7]:
"""
    these have 2 indexes summery index and vector index
    those are having a different retrival behaviours
    vector index--> as a text embeddings and its core anstraction and llama index and a core abstraction for building any sort of RAG system 
    summaru index--> quaring it will return all the nodes currently in the index 
        and it does'nt necessarily depend on the user query but will return all the nodes 
        that's currently in the index.
"""

from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

In [8]:
#Define Query Engines and Set Metadata

summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True, #enforce faster query generation by leveraging async capabilities
)
vector_query_engine = vector_index.as_query_engine()

In [10]:
from llama_index.core.tools import QueryEngineTool #query tool mean just the query engine with metadata


summary_tool = QueryEngineTool.from_defaults(#useful for summerization questions related to Meta GPT paper
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to MetaGPT"
    ),
)

vector_tool = QueryEngineTool.from_defaults(#useful for retriving specific context from the Meta GPT paper
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the MetaGPT paper."
    ),
)

In [ ]:
#Define Router Query Engine

In [11]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine #se
from llama_index.core.selectors import LLMSingleSelector # have various selectors please see screenshots


query_engine = RouterQueryEngine( #router query engine takes in a selector type as well as a set of query engine tools.
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [21]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: This choice indicates that the document is useful for summarization questions related to MetaGPT..
The document discusses the design and testing of a cooling system for a hybrid-powered vehicle that utilizes waste energy from exhaust gas and solar energy to power the electric compressor of the refrigeration system. It explores various refrigeration system cycles such as vapor compression, absorption, Stirling, and desiccant cycles as alternatives for vehicle cooling. The study aims to increase engine efficiency, reduce fuel consumption, and provide a sustainable solution for vehicle air conditioning systems by incorporating clean energy sources and waste heat recovery mechanisms.


In [13]:
print(len(response.source_nodes))

34


In [22]:
response = query_engine.query(
    "give me summery in point form?"
)
print(str(response))

Selecting query engine 0: Useful for summarization questions related to MetaGPT.
- Project focuses on designing a cooling system for a hybrid-powered vehicle using kinetic energy from exhaust gases and solar energy.
- System aims to reduce engine load, fuel consumption, and increase engine efficiency.
- Utilizes waste heat recovery and clean energy utilization to operate the electric compressor of the refrigeration system.
- Study shows improved engine efficiency and a 5.18% increase in the coefficient of performance compared to conventional systems.
- Various refrigeration cycles like vapor compression, absorption, Stirling, and desiccant are discussed as alternatives.
- Experimental setup involves components for solar energy conversion and recovering kinetic and thermal energy from the engine.
- System includes a diesel engine, photovoltaic cells, and a vapor compression system.
- Kia Carnival diesel engine is used due to its higher density and exhaust gas temperature.
- Photovoltaic

In [ ]:
#Getting Response using utils files and all the queries add to the utils file

In [18]:
#we can use helper function for getting router query engine and named utils file.we can add all the data like query tools selectors like that things to utils file and we can retrive data from utils

from utils import get_router_query_engine

query_engine = get_router_query_engine("Design of a cooling system for a hybrid-powered vehicle.pdf")

In [20]:
response = query_engine.query("Tell me about this pdf?")
print(str(response))

Selecting query engine 1: The question is asking for information about a specific PDF, which would require retrieving specific context from the MetaGPT paper..
The PDF document discusses the design of a cooling system for a hybrid-powered vehicle. It includes information on utilizing exhaust gas from an internal combustion engine to power a vapor compression cycle air conditioning system. The document also covers the efficiency of photovoltaic solar panels for the refrigeration system, as well as factors influencing the cooling level in the vehicle's cabin room. Additionally, it presents results and discussions related to the proposed cooling system, including diagrams illustrating the test rig and the relationship between cooling load, engine speed, and system performance.
